In [1]:
#Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Importing the dataset
dataset=pd.read_csv('Predicting-House-Prices-In-Bengaluru-Train-Data.csv')
X=dataset.iloc[:,[0,1,3,5,6,7]].values
y=dataset.iloc[:,8].values

In [2]:
#Handle missing data
from sklearn.preprocessing import Imputer
imputer=Imputer(missing_values='NaN',strategy='mean',axis=0)
imputer=imputer.fit(X[:,5:6])
X[:,5:6]=imputer.transform(X[:,5:6])
Xdf=pd.DataFrame(X)

In [3]:
#To remove the BHK or Bedroom words from column
Xdf[2]=Xdf[2].str.rstrip('BHKRedroom').astype('float')
#To replace dates with Under Construction
Xdf[1].loc[Xdf[1].str.contains('-')]='Under Construction'

C:\Users\Sarvesh\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
#Encoding categorical variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelEncoder_Xdf=LabelEncoder()
labelEncoder_Xdf.fit(Xdf.values[:,0])
Xdf[0]=labelEncoder_Xdf.transform(Xdf[0])

labelEncoder_Xdf1=LabelEncoder()
labelEncoder_Xdf1.fit(Xdf.values[:,1])
Xdf[1]=labelEncoder_Xdf1.transform(Xdf[1])

In [5]:
#Creating seperate columns i.e using Hot Encoding
oneHotEncoder=OneHotEncoder()
oneHotEncoder.fit(Xdf.values[:,0:1])
onehot=oneHotEncoder.transform(Xdf.values[:,0:1]).toarray()
Xdf=Xdf.join(pd.DataFrame(onehot),rsuffix='_0')
Xdf=Xdf.values[:,:9]
Xdf=pd.DataFrame(Xdf)
Xdf=Xdf.values[:,1:]
Xdf=pd.DataFrame(Xdf)

oneHotEncoder1=OneHotEncoder()
oneHotEncoder1.fit(Xdf.values[:,0:1])
onehot=oneHotEncoder1.transform(Xdf.values[:,0:1]).toarray()
Xdf=Xdf.join(pd.DataFrame(onehot[:,[0,1]]),rsuffix='_0')
Xdf=Xdf.values[:,1:]
Xdf=pd.DataFrame(Xdf)

In [6]:
#Renaming columns
Xdf.columns=['size','total_sqft','bath','balcony','built_up','carpet','plot','immediate','readymove']

#Xdf['total_sqft'].loc[Xdf['total_sqft'].str.contains('-')]
for i,element in enumerate(Xdf['total_sqft']):
    if '-' in element:
        element=element.replace(" ","")
        parts=element.split('-')
        element=str((float(parts[0])+float(parts[1]))/2)
        Xdf['total_sqft'][i]=element
    elif 'Yards' in element:
        element=element.replace('Sq. Yards','')
        element=str(float(element)*9)
        Xdf['total_sqft'][i]=element
    elif 'Meter' in element:
        element=element.replace('Sq. Meter','')
        element=str(float(element)*10.7639)
        Xdf['total_sqft'][i]=element
    elif 'Acres' in element:
        element=element.replace('Acres','')
        element=str(float(element)*43560)
        Xdf['total_sqft'][i]=element
    elif 'Guntha' in element:
        element=element.replace('Guntha','')
        element=str(float(element)*1089)
        Xdf['total_sqft'][i]=element
    elif 'Cents' in element:
        element=element.replace('Cents','')
        element=str(float(element)*435.61)
        Xdf['total_sqft'][i]=element
    elif 'Perch' in element:
        element=element.replace('Perch','')
        element=str(float(element)*272.25)
        Xdf['total_sqft'][i]=element
    elif 'Grounds' in element:
        element=element.replace('Grounds','')
        element=str(float(element)*2400)
        Xdf['total_sqft'][i]=element
#Deleting one row        
Xdf=Xdf.drop([4086])
y = np.delete(y, (4086), axis=0)

In [7]:
#Handle missing data
X1=Xdf.values
from sklearn.preprocessing import Imputer
imputer=Imputer(missing_values='NaN',strategy='mean',axis=0)
imputer=imputer.fit(X1[:,[0,2]])
X1[:,[0,2]]=imputer.transform(X1[:,[0,2]])
Xdf=pd.DataFrame(X1)

In [8]:
####################################Splitting and feature scaling###################
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xdf, y, test_size = 0.2, random_state = 0)

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train.reshape(-1,1))

C:\Users\Sarvesh\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
############################Appying Linear Regression######################
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(X_train,y_train)

y_pred = regressor.predict(X_test)
y_pred=sc_y.inverse_transform(y_pred)

In [10]:
####### Model Accuracy using RMSE#############
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(y_test,y_pred))

122.25019077868501

In [11]:
y_test

array([ 43.2, 117. ,  75. , ..., 100. , 395. , 290. ])

In [12]:
y_pred

array([[ 75.90855347],
       [126.45082011],
       [112.18731865],
       ...,
       [197.89709248],
       [265.56385048],
       [554.01028433]])